In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adm-lt-2024-2025-hackathon-rag/corpus.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/example_submission.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/train.csv
/kaggle/input/adm-lt-2024-2025-hackathon-rag/test.csv


In [4]:
# %%bash
# echo "🔧 Installing OpenJDK-21 ..."
# apt-get update -qq
# apt-get install -y --no-install-recommends openjdk-21-jdk-headless

# echo "🔧 Setting JAVA_HOME ..."
# echo 'export JAVA_HOME=/usr/lib/jvm/java-21-openjdk-amd64' >> ~/.bashrc
# echo 'export PATH=$JAVA_HOME/bin:$PATH' >> ~/.bashrc
# source ~/.bashrc

# java -version

🔧 Installing OpenJDK-21 ...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  openjdk-21-jre-headless
Suggested packages:
  openjdk-21-demo openjdk-21-source libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei | fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-21-jdk-headless openjdk-21-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 180 not upgraded.
Need to get 129 MB of archives.
After this operation, 299 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 openjdk-21-jre-headless amd64 21.0.7+6~us1-0ubuntu1~22.04 [46.8 MB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 openjdk-21-jdk-headless amd64 21.0.7+6~us1-0ubuntu1~22.04 [82.6 MB]
Fetched 129 MB in 5s (28.4 MB/s)
Selecting previously unselected package openjdk-21-jre-headless:

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
openjdk version "21.0.7" 2025-04-15
OpenJDK Runtime Environment (build 21.0.7+6-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 21.0.7+6-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [5]:
!pip -q install pyserini faiss-cpu sentence-transformers transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.3/195.3 MB 8.6 MB/s eta 0:00:000:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

In [6]:
import os, gc, json, re, pickle, numpy as np, pandas as pd, nltk, faiss, torch
nltk.download('punkt')

df_corpus = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/corpus.csv")

def clean(text):
    text = text.replace('\n', ' ').strip()
    text = re.sub(r"\s+", " ", text)
    return text

df_corpus["passage_clean"] = df_corpus["passage"].map(clean)
print(df_corpus.shape)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(40181, 3)


In [1]:
import torch, os
assert torch.cuda.is_available()

print("GPU:", torch.cuda.get_device_name(0))        

from sentence_transformers import SentenceTransformer

model_name  = "sentence-transformers/all-MiniLM-L6-v2"
dense_model = SentenceTransformer(model_name, device='cuda') 

GPU: Tesla T4


2025-06-03 17:35:06.091187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748972106.485665      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748972106.598359      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [8]:
from nltk.tokenize import sent_tokenize
chunks, ids = [], []

for pid, passage in zip(df_corpus.id, df_corpus.passage_clean):
    sents = sent_tokenize(passage)
    # group every 2 sentences
    for i in range(0, len(sents), 2):
        chunk = " ".join(sents[i:i+2])
        chunks.append(chunk)
        ids.append(f"{pid}_{i//2}")

corpus_chunks = pd.DataFrame({"chunk_id": ids, "text": chunks})
corpus_chunks.to_parquet("/kaggle/working/corpus_chunks.parquet")
print(corpus_chunks.shape)

(198651, 2)


In [9]:
from sentence_transformers import SentenceTransformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
dense_model = SentenceTransformer(model_name)

texts = corpus_chunks["text"].tolist()
emb = dense_model.encode(
        texts,
        batch_size=1024,         
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,
)
index = faiss.IndexHNSWFlat(emb.shape[1], 32)
index.hnsw.efConstruction = 200
index.add(emb.astype('float32'))

faiss.write_index(index, "/kaggle/working/dense.index")
pickle.dump(corpus_chunks["chunk_id"].tolist(), open("/kaggle/working/chunk_ids.pkl","wb"))

Batches:   0%|          | 0/194 [00:00<?, ?it/s]

In [10]:
import json, os, tqdm, pandas as pd, pathlib

DOCS_DIR = pathlib.Path("/kaggle/working/json_docs")
DOCS_DIR.mkdir(exist_ok=True)

for cid, text in tqdm.tqdm(zip(corpus_chunks.chunk_id, corpus_chunks.text),
                           total=len(corpus_chunks)):
    with open(DOCS_DIR/f"{cid}.json", "w") as f:
        json.dump({"id": str(cid), "contents": text}, f)

100%|██████████| 198651/198651 [00:14<00:00, 13736.40it/s]


In [11]:
%%bash
echo "Building BM25 index with Pyserini"

INPUT_DIR="/kaggle/working/json_docs"   
INDEX_DIR="/kaggle/working/bm25_index"

python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input "$INPUT_DIR" \
  --index "$INDEX_DIR" \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

📦 Building BM25 index with Pyserini …
2025-05-31 20:58:59,144 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:205) - Setting log level to INFO
2025-05-31 20:58:59,149 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - ============ Loading Index Configuration ============
2025-05-31 20:58:59,149 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) - AbstractIndexer settings:
2025-05-31 20:58:59,149 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + DocumentCollection path: /kaggle/working/json_docs
2025-05-31 20:58:59,150 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + CollectionClass: JsonCollection
2025-05-31 20:58:59,150 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Index path: /kaggle/working/bm25_index
2025-05-31 20:58:59,151 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Threads: 4
2025-05-31 20:58:59,151 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:214) -  + Opti

May 31, 2025 8:58:59 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [12]:
from pyserini.search.lucene import LuceneSearcher
bm25 = LuceneSearcher("/kaggle/working/bm25_index")
bm25.set_bm25(k1=0.9, b=0.4)

index = faiss.read_index("/kaggle/working/dense.index")
chunk_ids = pickle.load(open("/kaggle/working/chunk_ids.pkl","rb"))

def retrieve(question, k_bm25=60, k_dense=60):
    # BM25
    hits = bm25.search(question, k=k_bm25)
    bm25_ids = {h.docid for h in hits}
    # dense
    q_emb = dense_model.encode(question, normalize_embeddings=True)
    dist, idxs = index.search(np.expand_dims(q_emb,0), k_dense)
    dense_ids = {chunk_ids[i] for i in idxs[0]}
    # hybrid union
    merged_ids = list(bm25_ids | dense_ids)
    texts = corpus_chunks.set_index("chunk_id").loc[merged_ids].text.tolist()
    return merged_ids, texts


May 31, 2025 8:59:24 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


In [13]:
from sentence_transformers import CrossEncoder
rerank_ce = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank(question, passages, topn=10):
    pairs = [[question, p] for p in passages]
    scores = rerank_ce.predict(pairs, batch_size=32, convert_to_numpy=True)
    best = sorted(zip(scores, passages), reverse=True)[:topn]
    return [p for _,p in best]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import login
login("You need to login yourself ") 

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_cfg = BitsAndBytesConfig(load_in_4bit=True,
                             bnb_4bit_use_double_quant=True,
                             bnb_4bit_compute_dtype=torch.bfloat16)

llm_name = "mistralai/Mistral-7B-Instruct-v0.2"
tok = AutoTokenizer.from_pretrained(llm_name, token = True)
tok.pad_token = tok.eos_token
tok.padding_side = "left"
llm = AutoModelForCausalLM.from_pretrained(llm_name,
                                           quantization_config=bnb_cfg,
                                           device_map="auto",
                                           token = True)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [16]:
SYSTEM = ("You are a biomedical assistant. Answer concisely using only the information below.\n"
          "If the answer is a list, separate items with semicolons.\n"
          "Do NOT add information not present in passages.\n\n")

def make_prompt(q, passages):
    joined = "\n".join(passages)
    return SYSTEM + f"Question: {q}\n\nPassages:\n{joined}\n\nAnswer:\n"

@torch.inference_mode()
def generate_answer(prompt, max_tokens=48):
    inpt = tok(prompt, return_tensors="pt").to(llm.device)
    out = llm.generate(**inpt,
                       max_new_tokens=max_tokens,
                       pad_token_id=tok.eos_token_id,
                       do_sample=False)
    txt = tok.decode(out[0], skip_special_tokens=True)
    return txt.split("Answer:")[-1].strip()

In [17]:
def answer_one(question):
    ids, texts = retrieve(question)
    reranked = rerank(question, texts, topn=5)       
    prompt = make_prompt(question, reranked)
    return generate_answer(prompt)

In [41]:
from tqdm import tqdm

batch_size   = 16               
all_answers  = []

def one_prompt(q):
    ids, texts   = retrieve(q)                  
    passages     = rerank(q, texts, topn=3)     
    return make_prompt(q, passages)

batch_prompts, pending_ids = [], []
for qid, q in tqdm(zip(df_test.id, df_test.question), total=len(df_test)):
    batch_prompts.append(one_prompt(q))
    pending_ids.append(qid)

    if len(batch_prompts) == batch_size or qid == df_test.id.iloc[-1]:
        inputs  = tok(batch_prompts, return_tensors="pt", padding=True).to(llm.device)

        outputs = llm.generate(
            **inputs,
            max_new_tokens=48,
            do_sample=False,
            pad_token_id=tok.eos_token_id
        )
        decoded = tok.batch_decode(outputs, skip_special_tokens=True)

        # remove "Answer:" prefix if model adds it
        clean = [txt.split("Answer:")[-1].strip() for txt in decoded]
        all_answers.extend(clean)

        # reset batch
        batch_prompts, pending_ids = [], []

df_sub = pd.DataFrame({"id": df_test.id, "answer": all_answers})
df_sub.to_csv("submission.csv", index=False)
df_sub.head()

  0%|          | 0/1180 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 1/1180 [00:00<06:00,  3.27it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

100%|█████████▉| 1179/1180 [1:44:07<00:00,  1.35it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 1180/1180 [1:45:12<00:00,  5.35s/it]


,id,answer
0,1702,Alzheimer's disease.
1,3135,PCV13 consists of 13 serotype-specific polysac...
2,650,1. JASPAR\n2. MOTIF\n3. MEME\n4. ChIPSeeker\n5...
3,741,"Yes, the circadian clock is involved in riboso..."
4,2949,Cemiplimab is a human PD-1 monoclonal antibody...


In [42]:
from sklearn.metrics import f1_score

def token_f1(pred, gold):
    def bag(s): return re.sub(r"[^\w\s]","",s.lower()).split()
    p, g = bag(pred), bag(gold)
    if not p or not g:
        return 0.0
    common = len(set(p)&set(g))
    if common==0: return 0.0
    precision = common/len(p)
    recall    = common/len(g)
    return 2*precision*recall/(precision+recall)

sample = pd.read_csv("/kaggle/input/adm-lt-2024-2025-hackathon-rag/train.csv").sample(100, random_state=0)
f1s=[]
for q,a in zip(sample.question, sample.answer):
    pred = answer_one(q)
    f1s.append(token_f1(pred, a))
print("Sample F1:", np.mean(f1s))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Sample F1: 0.32594382531288757
